In [1]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from IPython.display import display

In [2]:
df = pd.read_parquet('/Users/daviddangol/Documents/monthly report/domestic airlines/data/query-results_query-with-param_hotel_airlines_intl_bq_param_airlines_only_10-15-2025_11-01-2025000000000000.parquet')
display(df.columns)
display(df.info())

Index(['user_id', 'event_name', 'event_date', 'event_timestamp',
       'event_params', 'is_active_user', 'medium', 'source', 'platform',
       'version', 'install_source'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542684 entries, 0 to 542683
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   user_id          542336 non-null  object
 1   event_name       542684 non-null  object
 2   event_date       542684 non-null  object
 3   event_timestamp  542684 non-null  int64 
 4   event_params     542684 non-null  object
 5   is_active_user   0 non-null       object
 6   medium           537016 non-null  object
 7   source           537031 non-null  object
 8   platform         542684 non-null  object
 9   version          542684 non-null  object
 10  install_source   542684 non-null  object
dtypes: int64(1), object(10)
memory usage: 45.5+ MB


None

In [3]:
display(df['event_name'].unique())

array(['domesticFlightDetail', 'domesticFlightSearch',
       'domesticFlightConfirmation', 'domesticFlightPassengerDetail'],
      dtype=object)

In [4]:
df['event_name'].value_counts()

event_name
domesticFlightDetail             391751
domesticFlightSearch             142145
domesticFlightConfirmation         8723
domesticFlightPassengerDetail        65
Name: count, dtype: int64

In [5]:
funnel_data = {
    "Stage": ["domesticFlightDetail", "domesticFlightSearch", "domesticFlightConfirmation", "domesticFlightPassengerDetail"],
    "Count": [
        len(df[df['event_name'] == 'domesticFlightDetail']),
        len(df[df['event_name'] == 'domesticFlightSearch']),
        len(df[df['event_name'] == 'domesticFlightConfirmation']),
        len(df[df['event_name'] == 'domesticFlightPassengerDetail'])    ]
}

In [ ]:
!pip install plotly



  Using cached plotly-6.3.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached narwhals-2.10.1-py3-none-any.whl.metadata (11 kB)
Using cached plotly-6.3.1-py3-none-any.whl (9.8 MB)
Using cached narwhals-2.10.1-py3-none-any.whl (419 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]2m1/2 [plotly]


In [13]:
import plotly.graph_objects as go

funnel_df = pd.DataFrame(funnel_data)

# Calculate drop-off percentages
dropoff_percentages = [0] # Drop-off from a hypothetical stage before the first is 0
for i in range(1, len(funnel_df['Count'])):
    dropoff = funnel_df['Count'][i-1] - funnel_df['Count'][i]
    dropoff_percent = (dropoff / funnel_df['Count'][i-1]) * 100 if funnel_df['Count'][i-1] > 0 else 0
    dropoff_percentages.append(dropoff_percent)

# Create a list of text labels, including drop-off percentages
text_labels = []
for i, stage in enumerate(funnel_df['Stage']):
    conversion_rate = (funnel_df['Count'][i]/funnel_df['Count'][0]) * 100 if funnel_df['Count'][0] > 0 else 0
    if i == 0:
        text_labels.append(f"{funnel_df['Count'][i]} ({conversion_rate:.1f}%)")
    else:
         text_labels.append(f"{funnel_df['Count'][i]} ({conversion_rate:.1f}%) - {dropoff_percentages[i]:.1f}% Drop-off")


# Plot funnel chart using Plotly
fig = go.Figure(go.Funnel(
    y=funnel_df['Stage'],  # Y-axis represents the stages of the funnel
    x=funnel_df['Count'],  # X-axis represents the number of users at each stage
    textinfo="text",  # Show the custom text labels
    text=text_labels, # Assign the custom text labels
    textposition="auto" # Position the text outside the bars
))

# Update layout with a title and size
fig.update_layout(
    title="User Activation & Purchase Funnel with Drop-off Analysis",
    autosize=False, # Disable autosize
    width=1000, # Set the width of the chart
    height=900 # Set the height of the chart
)

# Show the interactive funnel chart
fig.show()